In [2]:
########### Python 3.2 #############
import requests
import json
import pandas as pd
import datetime as dt

############

In [2]:
!pip install ecmwf-api-client

In [6]:
url = "https://api.energyville.be/smarthordc/v1/"
source = 'Weather.Forecast.Rebase'
params = {
    'start': '2023-08-01T00:00:00Z',
    'end': '2023-08-31T00:00:00Z',
    'keys': ['Temperature', 'CloudCover', 'SolarDownwardRadiation', 'SurfaceDownSolarDiffuseRadiation', 'SurfaceDownSolarDirectRadiation', 'WindSpeed', 'RelativeHumidity']
}
body = {
    "Location": ["EnergyVille 1, BE"]
}
headers = {
    'Ocp-Apim-Subscription-Key': '1e3353b6342142cd96b61451416bd9cb',
}
response = requests.post(f'{url}/api/sources/{source}/timeseries?', headers=headers, params=params, json=body)
df = pd.DataFrame(json.loads(response.text))
df.head()

,Timestamp,TimeZoneOffset,MeasurementId,Value,Unit,Key,ForecastGeneratedTimestamp,ForecastWindow,ForecastStart,Latitude,Longitude,MetaData,Location
0,2023-08-01T00:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Surf...,38.71,W/m2,SurfaceDownSolarDirectRadiation,2023-07-30T00:00:01.4882788Z,2.00:00:00,2023-07-30T00:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
1,2023-08-01T00:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Surf...,32.68,W/m2,SurfaceDownSolarDiffuseRadiation,2023-07-31T19:00:01.2529782Z,2.00:00:00,2023-07-31T19:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
2,2023-08-01T00:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Clou...,1.00,,CloudCover,2023-07-30T04:00:01.3521798Z,2.00:00:00,2023-07-30T04:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
3,2023-08-01T00:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Clou...,1.00,,CloudCover,2023-07-30T02:00:01.3762791Z,2.00:00:00,2023-07-30T02:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
4,2023-08-01T00:00:00Z,,Weather.Forecast.Rebase.50994716-05539433.Wind...,4.05,m/s,WindSpeed,2023-07-31T13:00:02.0152034Z,2.00:00:00,2023-07-31T13:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"


In [7]:
#Only use the forecast at 11 o clock
# Convert ForecastGeneratedTimestamp to datetime if it's not already in datetime format
df['ForecastGeneratedTimestamp'] = pd.to_datetime(df['ForecastGeneratedTimestamp'])
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Round the ForecastGeneratedTimestamp to the nearest hour
df['ForecastGeneratedTimestamp'] = df['ForecastGeneratedTimestamp'].dt.round('H')

# Select rows with '11:00:00' in RoundedForecastGeneratedTimestamp
DA_forecast_datetimes = pd.to_datetime(df['Timestamp'].dt.date - pd.Timedelta(days=1), utc = True) + pd.Timedelta(hours=11)
DA_forecasts = df[df['ForecastGeneratedTimestamp'] == DA_forecast_datetimes]

# Drop unnecessary columns
#closest_forecasts = closest_forecasts.drop('RoundedForecastGeneratedTimestamp', axis=1)

# Display or use the resulting DataFrame (closest_forecasts)
DA_forecasts.head()


,Timestamp,TimeZoneOffset,MeasurementId,Value,Unit,Key,ForecastGeneratedTimestamp,ForecastWindow,ForecastStart,Latitude,Longitude,MetaData,Location
73,2023-08-01 00:00:00+00:00,,Weather.Forecast.Rebase.50994716-05539433.Wind...,4.05,m/s,WindSpeed,2023-07-31 11:00:00+00:00,2.00:00:00,2023-07-31T11:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
98,2023-08-01 00:00:00+00:00,,Weather.Forecast.Rebase.50994716-05539433.Surf...,56.80,W/m2,SurfaceDownSolarDiffuseRadiation,2023-07-31 11:00:00+00:00,2.00:00:00,2023-07-31T11:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
100,2023-08-01 00:00:00+00:00,,Weather.Forecast.Rebase.50994716-05539433.Temp...,16.62,degC,Temperature,2023-07-31 11:00:00+00:00,2.00:00:00,2023-07-31T11:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
127,2023-08-01 00:00:00+00:00,,Weather.Forecast.Rebase.50994716-05539433.Clou...,1.00,,CloudCover,2023-07-31 11:00:00+00:00,2.00:00:00,2023-07-31T11:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"
129,2023-08-01 00:00:00+00:00,,Weather.Forecast.Rebase.50994716-05539433.Sola...,-0.05,W/m2,SolarDownwardRadiation,2023-07-31 11:00:00+00:00,2.00:00:00,2023-07-31T11:00:00Z,50.994716,5.539433,"{'latitude': 50.994716, 'longitude': 5.539433}","EnergyVille 1, BE"


In [8]:
DA_forecasts = DA_forecasts.pivot(columns='Key', index='Timestamp', values='Value')

In [9]:
DA_forecasts.head()

Key,CloudCover,RelativeHumidity,SolarDownwardRadiation,SurfaceDownSolarDiffuseRadiation,SurfaceDownSolarDirectRadiation,Temperature,WindSpeed
Timestamp,,,,,,,
2023-08-01 00:00:00+00:00,1.00,90.19,-0.05,56.80,16.18,16.62,4.05
2023-08-01 01:00:00+00:00,1.00,88.74,-0.02,53.81,15.32,15.97,4.05
2023-08-01 02:00:00+00:00,1.00,89.85,0.03,51.12,14.56,15.49,3.50
2023-08-01 03:00:00+00:00,0.93,89.64,-0.02,48.69,13.86,15.19,3.56
2023-08-01 04:00:00+00:00,0.85,90.18,15.86,46.47,13.23,14.94,3.65


In [ ]:
import pickle
with open('Rebase.pickle', 'wb') as f:
    pickle.dump(DA_forecasts, f, pickle.HIGHEST_PROTOCOL)

ValueError: unrecognized engine cfgrib must be one of: ['scipy', 'store']

SyntaxError: invalid syntax (2874371567.py, line 1)